Find eigenvalue and eigenvectors (Book 1, p. 118, ex. 13.8).

In [1]:
import numpy as np
from numpy.linalg import eig, norm

import pandas as pd

# Common parts

In [2]:
# Initial test matrix.
A = np.array([[-0.81417, -0.01937,  0.41372],
              [-0.01937,  0.54414,  0.00590],
              [ 0.41372,  0.00590, -0.81445]])

In [3]:
def eigenvalue_find(A, v):
    Av = A.dot(v)
    return v.dot(Av)

def aposterior_error(A, eigenvalue, eigenvalue_predicted, eigenvector_predicted):
    """Calculating aposterior error of eigenvalue finding. 
    
    Args:
        A (ndarray<ndarray, ndarray>): matrix to handle.
        eigenvalue (float): true eigenvalue. 
        eigenvalue_predicted (float): eigenvalue we want to test. 
        eigenvector_predicted (ndarray<float>): eigenvector that corresponds to eigenvalu we want to test. 
    
    Returns:
        error (float): aposterior error. 
    """
    error = (norm(np.dot(A, eigenvector_predicted) - np.dot(eigenvalue_predicted, eigenvector_predicted)) /
             norm(eigenvector_predicted))
    return error

# Jacobi method

In [4]:
def jacobi_eigenvalue(A, eps=1e-6):
    """Computes the eigenvalues and eigenvectors using Jacobi rotation method. 
    
    Args:
        A (ndarray<ndarray, ndarray>): matrix to handle.
        eps (float): error rate.

    Returns:
        d (ndarray): eigenvalues.
        v (ndarray<ndarray, ndarray>): eigenvectors.
        it_num (int): number of iterations.
    """
  
    n = A.shape[0]
    
    v = np.zeros([n, n])
    d = np.zeros(n)
    
    for j in range(0, n):
        for i in range(0, n):
            v[i,j] = 0.0
        v[j,j] = 1.0
  
    for i in range(0, n):
        d[i] = A[i,i]
  
    bw = np.zeros(n)
    zw = np.zeros(n)
    w = np.zeros(n)
  
    for i in range(0, n):
        bw[i] = d[i]
  
    it_num = 0
    rot_num = 0
    
    thresh = 0.0
    for j in range(0, n):
        for i in range(0, j):
            if np.abs(A[i, j]) > thresh:
                thresh = np.abs(A[i, j])
  
    while (thresh >= eps):
  
        it_num += 1
    
        thresh = 0.0
        for j in range(0, n):
            for i in range(0, j):
                if np.abs(A[i, j]) > thresh:
                    thresh = np.abs(A[i, j])
  
        for p in range(0, n):
            for q in range(p + 1, n):
                
                gapq = 10.0 * abs (A[p, q])
                termp = gapq + abs (d[p])
                termq = gapq + abs (d[q])
  
                # Annihilate tiny offdiagonal elements.
                if (4 < it_num and termp == abs (d[p]) and termq == abs (d[q])):
                    A[p, q] = 0.0
    
                # Other  wise, apply a rotation.
                elif (thresh <= abs (A[p, q])):
                    h = d[q] - d[p]
                    term = abs (h) + gapq
  
                    if (term == abs (h)):
                        t = A[p, q] / h
                    else:
                        theta = 0.5 * h / A[p, q]
                        t = 1.0 / (abs (theta) + np.sqrt (1.0 + theta * theta))
                        if (theta < 0.0):
                            t = - t
  
                    c = 1.0 / np.sqrt (1.0 + t * t)
                    s = t * c
                    tau = s / (1.0 + c)
                    h = t * A[p, q]
    
                    # Accumulate corrections to diagonal elements.
                    zw[p] = zw[p] - h                  
                    zw[q] = zw[q] + h
                    d[p] = d[p] - h
                    d[q] = d[q] + h
  
                    A[p, q] = 0.0
    
                    # Rotate, using information from the upper triangle of A only.
                    for j in range(0, p):
                        g = A[j, p]
                        h = A[j, q]
                        A[j, p] = g - s * (h + g * tau)
                        A[j, q] = h + s * (g - h * tau)
  
                    for j in range(p + 1, q):
                        g = A[p, j]
                        h = A[j, q]
                        A[p, j] = g - s * (h + g * tau)
                        A[j, q] = h + s * (g - h * tau)
  
                    for j in range(q + 1, n):
                        g = A[p, j]
                        h = A[q, j]
                        A[p, j] = g - s * (h + g * tau)
                        A[q, j] = h + s * (g - h * tau)
    
                    # Accumulate information in the eigenvector matrix.
                    for j in range(0, n):
                        g = v[j, p]
                        h = v[j, q]
                        v[j, p] = g - s * (h + g * tau)
                        v[j, q] = h + s * (g - h * tau)
  
                    rot_num = rot_num + 1
  
        for i in range(0, n):
            bw[i] = bw[i] + zw[i]
            d[i] = bw[i]
            zw[i] = 0.0
    
    # Restore upper triangle of input matrix.
    for j in range(0, n):
        for i in range(0, j):
            A[i, j] = A[j, i]
  
    # Ascending sort the eigenvalues and eigenvectors.
    for k in range(0, n - 1):
        m = k
        for l in range(k + 1, n):
            if (d[l] < d[m]):
                m = l
  
        if (k != m):
            t    = d[m]
            d[m] = d[k]
            d[k] = t
  
            for i in range(0, n):
                w[i]   = v[i, m]
                v[i, m] = v[i, k]
                v[i, k] = w[i]
  
    return d, v, it_num

### Jacobi method test

In [5]:
values, vectors, it_num = jacobi_eigenvalue(A)

print("\nEigenvalues: \n{}".format(values))
print("\nEigenvectors: \n{}".format(vectors))
print("\nNumber of iterations: \n{}".format(it_num))


Eigenvalues: 
[-1.22821015 -0.40068602  0.54441617]

Eigenvectors: 
[[-7.07033278e-01  7.07036576e-01 -1.42556742e-02]
 [-1.00810634e-02  1.00795044e-02  9.99898383e-01]
 [ 7.07108419e-01  7.07105144e-01  1.13550152e-06]]

Number of iterations: 
5


# Power iterations method.

In [6]:
def power_iteration(A, k):
    """Calculating dominant eigenvalue and eigenvector using power iteration method. 
    
    Args:
        A (ndarray<ndarray, ndarray>): matrix to handle.
        k (int): number of iterations.
        
    Returns:
        ev_new (float): dominant eigenvalue. 
        v_new (ndarray<float>): dominant eigenvector. 
    """
    n = A.shape[0]

    v = np.ones(n) / np.sqrt(n)
    ev = eigenvalue_find(A, v)

    for i in range(k):

        Av = A.dot(v)

        v_new = Av / np.linalg.norm(Av)
        ev_new = eigenvalue_find(A, v_new)

        v = v_new
        ev = ev_new

    return ev_new, v_new

In [7]:
def power_iteration_stats(A, eigenvalue, eps=1e-3):
    """Calculating number of iterations to reach given aposterior error.
    
    Args:
        A (ndarray<ndarray, ndarray>): matrix to handle.
        eigenvalue (float): true eigenvalue.
        eps (float): error to reach.
        
    Returns:
         k (int): number of iteraions to reach eps error.
         error (float): result aposterior error.
    """

    k = 1
    
    eigenvalue_predicted, eigenvector_predicted = power_iteration(A, k)
    error = aposterior_error(A, eigenvalue, eigenvalue_predicted, eigenvector_predicted)
    
    while (error > eps):
        k  += 1
        
        eigenvalue_predicted, eigenvector_predicted = power_iteration(A, k)
        error = aposterior_error(A, eigenvalue, eigenvalue_predicted, eigenvector_predicted)
        
    return k, error

### Power iterations method test

In [8]:
eigenvalue = eig(A)[0][np.argmax(np.abs(eig(A)[0]))]

k, error = power_iteration_stats(A, eigenvalue)
dominant_eigenvalue, dominant_eigenvector = power_iteration(A, k)

print("\nDominant eigenvalue: \n{}".format(dominant_eigenvalue))
print("\nDominant eigenvector: \n{}".format(dominant_eigenvector))
print("\nNumber of iterations to reach given aposterior error: \n{}".format(k))
print("\nResult error: \n{}".format(error))


Dominant eigenvalue: 
-1.2282097191128667

Dominant eigenvector: 
[-0.70704531 -0.00958763  0.70710325]

Number of iterations to reach given aposterior error: 
15

Result error: 
0.0008749131534185878
